# Basic RAG System with LangChain




### Load Dependencies

In [1]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.22
    Uninstalling langchain-core-0.3.22:
      Successfully uninstalled langchain-core-0.3.22
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.10
    Uninstalling langchai

In [2]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.6 MB/s eta 0:00:

## Enter API Tokens

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter your OpenAI Key: ')

Enter your OpenAI Key: ··········


## Setup Environment Variables

In [4]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Load Wikipedia Data

In [5]:
import gzip
import json
import requests
from tqdm import tqdm
import sys
import os

# Download a file from a URL
def http_get(url, path) -> None:
    """
    Downloads a URL to a given path on disc
    """
    if os.path.dirname(path) != "":
        os.makedirs(os.path.dirname(path), exist_ok=True)

    req = requests.get(url, stream=True)
    if req.status_code != 200:
        print("Exception when trying to download {}. Response {}".format(url, req.status_code), file=sys.stderr)
        req.raise_for_status()
        return

    download_filepath = path + "_part"
    with open(download_filepath, "wb") as file_binary:
        content_length = req.headers.get("Content-Length")
        total = int(content_length) if content_length is not None else None
        progress = tqdm(unit="B", total=total, unit_scale=True)
        for chunk in req.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                progress.update(len(chunk))
                file_binary.write(chunk)

    os.rename(download_filepath, path)
    progress.close()


wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

100%|██████████| 50.2M/50.2M [00:15<00:00, 3.15MB/s]


In [6]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        #Only add the first paragraph
        passages.append(data['paragraphs'][0])

print("Passages:", len(passages))

Passages: 169597


In [7]:
# We subset our data so we only use a subset of wikipedia to run things faster
passages = [passage for passage in passages for x in ['fish', 'india', 'cheetah']
              if x in passage.lower().split()]

In [8]:
len(passages)

1070

In [9]:
passages[0]

"An electric fish is a fish that can make electric fields. Many fish (sharks, rays, and catfish) can detect electric fields: they are 'electroreceptive'. They cannot make electric fields, and are not called electric fish."

### Load Open AI LLMs

In [10]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

### Generate LLM Embeddings and store them in Chroma Vector DB

**Chroma Vector DB** is a versatile, open-source vector database designed for managing and querying vector embeddings. It is easy to set up and integrates well with various AI tools and algorithms. Chroma is particularly useful for applications that require rapid and precise retrieval of content represented as embeddings—efficient data formats for text, images, and soon, audio and video.

**Key Features:**
- **Integration with AI Tools:** Chroma supports embedding functions from leading providers like OpenAI, Google, and Hugging Face, allowing for flexible and powerful data handling.
- **Ease of Use:** The database provides default embedding functions, or users can integrate external APIs to generate embeddings.
- **Efficient Querying:** Users can create collections to store embeddings, documents, and metadata. These can be queried to retrieve the most similar items, making information retrieval quick and effective.
- **Flexible API:** Chroma offers a straightforward API that supports both standard operations and custom embedding functions.

For more detailed information, visit the official Chroma documentation [here](https://docs.trychroma.com).


In [11]:
passages[:3]

["An electric fish is a fish that can make electric fields. Many fish (sharks, rays, and catfish) can detect electric fields: they are 'electroreceptive'. They cannot make electric fields, and are not called electric fish.",
 'Pilot fish ("Naucrates ductor") are fish that live in many places of the world. They live in warm water. They eat parasites on larger fish.',
 'Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silky leaves 3–5\xa0cm long and 1–3\xa0cm broad. The leaves are opposite each other. The flowers are quite big.

In [12]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [13]:
# The vectorstore we'll be using
from langchain_chroma import Chroma

# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc) for doc in passages]

In [15]:
docs[:3]

[Document(metadata={}, page_content="An electric fish is a fish that can make electric fields. Many fish (sharks, rays, and catfish) can detect electric fields: they are 'electroreceptive'. They cannot make electric fields, and are not called electric fish."),
 Document(metadata={}, page_content='Pilot fish ("Naucrates ductor") are fish that live in many places of the world. They live in warm water. They eat parasites on larger fish.'),
 Document(metadata={}, page_content='Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silk

In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000,
                                          chunk_overlap=200)
chunked_docs = splitter.split_documents(docs)

In [17]:
chunked_docs[:3]

[Document(metadata={}, page_content="An electric fish is a fish that can make electric fields. Many fish (sharks, rays, and catfish) can detect electric fields: they are 'electroreceptive'. They cannot make electric fields, and are not called electric fish."),
 Document(metadata={}, page_content='Pilot fish ("Naucrates ductor") are fish that live in many places of the world. They live in warm water. They eat parasites on larger fish.'),
 Document(metadata={}, page_content='Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silk

## Create Vector DB and Retriever

If you have already created `wiki_db`in the previous hands-on session then just load the DB and DO NOT run the following code to create the database again, ignore this when running on Colab

In [18]:
# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs, collection_name='wiki_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wiki_db")

## Load Vector DB from disk

Run the following code if your vector DB already exists on disk from the previous hands-on session

In [19]:
# load from disk
chroma_db = Chroma(persist_directory="./wiki_db",
                   collection_name='wiki_db',
                   embedding_function=openai_embed_model)

In [20]:
chroma_db

In [21]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 5, "score_threshold": 0.2})

### Build a QA RAG Chain

In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [23]:
prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point.

            Question:
            {question}

            Context:
            {context}

            Answer:
         


## RAG Chain - Using LCEL

In [24]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    prompt_template
      |
    chatgpt
)

In [25]:
query = "What is the fastest animal?"
result = qa_rag_chain.invoke(query)
print(result.content)

The fastest animal is the cheetah, which can run up to 112 kilometers per hour.


In [26]:
query = "Who was the winner of the champions league in 2020?"
result = qa_rag_chain.invoke(query)
print(result.content)

I don't know.


In [27]:
query = "What is the capital of India?"
result = qa_rag_chain.invoke(query)
print(result.content)

The capital of India is New Delhi.


In [28]:
query = "Tell me more about it in detail"
result = qa_rag_chain.invoke(query)
print(result.content)

I don't know.


# Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

This guide focuses on integrating historical messages into the application's logic. Additional details on managing chat history can be found [here](https://python.langchain.com/docs/expression_language/how_to/message_history/).

![](https://i.imgur.com/8hLJMPl.gif)

### Building on the Q&A RAG System - to a Conversational Q&A RAG System

We will enhance our Q&A RAG System, which utilizes the Wikipedia dataset, by implementing the following updates:

- **Prompt Adjustment:** Our prompt will be modified to include historical messages as inputs, allowing the system to maintain context over the course of a conversation.

- **Contextualizing Questions:** We will introduce a sub-chain mechanism to reformulate the latest user query by considering the chat history. This is crucial for understanding questions that refer back to previous messages. For example, a query like "Can you elaborate on the second point?" relies on the context provided by preceding interactions, which affects the system's ability to retrieve relevant information effectively.





## Contextualizing the Question

To maintain a seamless flow in conversations, especially in a Q&A setting, it's essential to incorporate historical interactions. Here’s how we achieve this:

### Defining a Sub-Chain for Historical Context

1. **Sub-Chain Creation:** We'll define a sub-chain that uses both historical messages and the latest user query. This sub-chain reformulates the question if it refers to any past interactions, ensuring the system, especially the vector database understands the context to return the most relevant documents to this newly reworded question.

2. **Using `MessagesPlaceholder`:** Our prompt construction involves a `MessagesPlaceholder` variable named `chat_history`. This setup allows us to input a list of messages using the `chat_history` key. The system integrates these messages, positioning them after its own responses and before the latest user question.

3. **Helper Function Usage:** We employ the `create_history_aware_retriever` function available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html). This function is crucial for handling instances where the chat history might be empty and orchestrates the sequence of operations: `prompt | llm | StrOutputParser() | retriever`.

4. **Chain Construction:** The `create_history_aware_retriever` constructs a chain that processes inputs under the keys `input` and `chat_history`, ensuring the output schema aligns with that of a retriever.

By implementing these steps, our system can effectively utilize historical context to better understand and respond to user queries, thereby enhancing the conversational experience.


In [29]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)

This chain prepends a rephrasing of the input query to our retriever, so that the retrieval incorporates the context of the conversation.

## Building the QA RAG Chain with Chat History

Now we're ready to construct our comprehensive QA RAG chain, which leverages historical context for more accurate and relevant responses.

### Components of the QA RAG Chain

1. **Creating Document Chains:**
   - We use the `create_stuff_documents_chain` function, which is detailed [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html). This function is used to create a `question_answer_chain`, accepting inputs such as `context`, `chat_history`, and `input`. It efficiently combines the retrieved context with the conversation history and the current query to generate an informed answer.

2. **Building the Final QA RAG Chain:**
   - The entire QA RAG chain is assembled using the `create_retrieval_chain` function, available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html). This chain integrates the `history_aware_retriever` with the `question_answer_chain`. It retains intermediate outputs like the retrieved context for added convenience during the query handling process.
   - The `create_retrieval_chain` function accepts keys such as `input` and `chat_history` and includes `input`, `chat_history`, `context`, and `answer` in its outputs.

By implementing these steps, the system not only contextualizes but also provides accurate answers by synthesizing information from both the current and historical interactions. This method enhances the conversational AI’s ability to understand and respond to user queries dynamically, making the interactions more engaging and relevant.


In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If the answer is not present in the context, just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}

                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [31]:
chat_history = []

question = "What is the capital of India?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

The capital of India is New Delhi.


In [32]:
chat_history

[]

In [33]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={})]

In [34]:
question = "Tell me more about this city"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

New Delhi is the capital of India and a union territory of the megacity of Delhi. It has a rich history and is home to several monuments. The city is known for being expensive to live in and falls under the North Indian geographical zone. New Delhi covers an area of about 42.7 km and has a population of approximately 9.4 million people.


In [ ]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me more about this city', additional_kwargs={}, response_metadata={}),
 AIMessage(content='New Delhi is the capital of India and a union territory of the megacity of Delhi. It has a rich history and is home to several monuments. The city is known for being expensive to live in and falls under the North Indian geographical zone. New Delhi covers an area of about 42.7 km and has a population of approximately 9.4 million people.', additional_kwargs={}, response_metadata={})]

In [ ]:
question = "What is the fastest animal?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

The fastest land animal is the cheetah, which can run up to 112 kilometers per hour for a short time.


In [ ]:
response

{'input': 'What is the fastest animal?',
 'chat_history': [HumanMessage(content='What is the capital of India?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Tell me more about this city', additional_kwargs={}, response_metadata={}),
  AIMessage(content='New Delhi is the capital of India and a union territory of the megacity of Delhi. It has a rich history and is home to several monuments. The city is known for being expensive to live in and falls under the North Indian geographical zone. New Delhi covers an area of about 42.7 km and has a population of approximately 9.4 million people.', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs li

In [ ]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])

In [ ]:
chat_history

[HumanMessage(content='What is the capital of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me more about this city', additional_kwargs={}, response_metadata={}),
 AIMessage(content='New Delhi is the capital of India and a union territory of the megacity of Delhi. It has a rich history and is home to several monuments. The city is known for being expensive to live in and falls under the North Indian geographical zone. New Delhi covers an area of about 42.7 km and has a population of approximately 9.4 million people.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the fastest animal?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The fastest land animal is the cheetah, which can run up to 112 kilometers per hour for a short time.', additional_kwargs={}, response_metadata={})]

In [ ]:
chat_history[-2:]

[HumanMessage(content='What is the fastest animal?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The fastest land animal is the cheetah, which can run up to 112 kilometers per hour for a short time.', additional_kwargs={}, response_metadata={})]

In [ ]:
question = "Tell me about its different species"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
print(response['answer'])

There are several subspecies of cheetah:

1. **South African Cheetah (Acinonyx jubatus jubatus)**: This is the nominate subspecies native to Southern Africa and is the most abundant, with an estimated population of more than 6,000 individuals in the wild.

2. **Asiatic Cheetah (Acinonyx jubatus venaticus)**: This subspecies is critically endangered and is native to Asia.

These subspecies differ in their geographical distribution and population status.


# Multi-User Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

However in most real-world conversational systems, multiple users or user sessions will be accessing the system simultaneously.

![](https://i.imgur.com/X4WivLu.gif)

Here we will show how you can use `SQLChatMessageHistory` such that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`




In [35]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

######### REPHRASER ############
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)



######### MULTI_USER RAG RESPONSE GENERATOR ############
qa_system_prompt = """You are an assistant for question-answering tasks.
                      Analyze the user question carefully and use the following pieces of
                      retrieved context to answer the question.
                      If the answer is not present in the context, just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}
                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    history = SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    return history

# subset historical conversations based on last K conversation messages
# here by default we use the last 10 conversations (ai-human) as memory to the input prompt
def memory_buffer_window(messages, lastk_conversations=10):
    return messages[-(lastk_conversations*2):] # each conversation has 2 messages - (human prompt, AI response)

# custom RAG chain which looks at last K conversational messages
question_answer_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: memory_buffer_window(x["chat_history"]))
      |
    qa_prompt
      |
    chatgpt
      |
    StrOutputParser()
)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


############ CONVERSATIONAL RAG CHAIN ####################
conversational_rag_chain = RunnableWithMessageHistory(
    qa_rag_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [37]:
from IPython.display import display, Markdown

def conv_rag_chatbot(usersession_id, prompt):
    response = conversational_rag_chain.invoke(
                                {"input": prompt},
                                config={
                                    "configurable": {"session_id": usersession_id}
                                }
    )
    print('Answer:')
    display(Markdown(response['answer']))
    print('Sources:')
    for document in response['context']:
        print(document)
        print()

    return response

In [38]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'What is the capital of India?')

/usr/local/lib/python3.10/dist-packages/langchain_core/runnables/history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Answer:


The capital of India is New Delhi.

Sources:
page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.'

page_content='The Republic of India is divided into twenty-eight States,and eight union territories including the National Capital Territory.'

page_content='Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 19

In [39]:
r = conv_rag_chatbot(us_id, 'Tell me more about it')

Answer:


New Delhi is the capital of India and a union territory of the megacity of Delhi. It has a rich history and is home to several monuments. The city is known for being expensive to live in and falls under the North Indian zone in traditional geography. New Delhi covers an area of about 42.7 km and has a population of approximately 9.4 million people.

Sources:
page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.'

page_content='Rajpath (, meaning "King's Way") is the national boulevard of India. It is in New Delhi, the capital of India. The boulevard starts at the home of the President of India and ends at the National Stadium.'

page_content='Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nat

In [40]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'What is the fastest animal on land?')

Answer:


The fastest land animal is the cheetah, which can run up to 112 kilometers per hour for a short time.

Sources:
page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.'

page_content='The South African Cheetah ("Acinonyx jubatus jubatus"), also known as Namibian Cheetah, is the nominate subspecies of cheetah native to Southern Africa. It is the most abundant subspecies estimated at more than 6,000 individuals in the wild. Since 1990 and onwards, the population was estimated at approximately 2,500 individuals in Namibia, until 2015, the cheetah population has been increased to more than 3,500 in the country.'

page_content='Leedsichthys problematicus ("leeds fish") was a giant fossil fish of the Jurassic period. It was a pachycormid, a group of extinct ray-finned fishes (Actinopterygii). "Leedsichthys" is the large

In [41]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'Tell me about wildlife in India')

Answer:


India is home to a diverse range of wildlife, including several notable species. The Bengal tiger, native to the Indian subcontinent, is one of the most iconic animals and is the national animal of both India and Bangladesh. The Indian elephant, another significant species, is listed as endangered and faces threats from habitat loss and fragmentation. Additionally, the Bengal fox is endemic to the region. India has established national parks and wildlife protection laws, such as the Wildlife Protection Act of 1972, to conserve its rich biodiversity.

Sources:
page_content='The Bengal tiger ("Panthera tigris tigris") is a tiger subspecies native to the Indian subcontinent. It lives in Bhutan, Nepal, Bangladesh and northern India. It is the National animal of both India and Bangladesh.'

page_content='The Wildlife Protection Act of 1972 is a set of laws created in 1972 by the Government of India. Before 1972, India had only five national parks.'

page_content='The Bengal Fox ("Vulpes bengalensis"), also known as the Indian Fox, is a fox. It is endemic to the Indian subcontinent, and is found from the Himalayan foothills and Terai of Nepal through southern India and from southern and eastern Pakistan to eastern India and southeastern Bangladesh.'

page_content='There are lots of different varieties of Indian music. Most of the different types of music are similar to Western music types. Pop, classical, and folk are all included. India’s classical music has a history originating hundreds of years back and is still popular in India toda

In [ ]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'tell me more about its different species')

Answer:


There are several subspecies of cheetah:

1. **South African Cheetah ("Acinonyx jubatus jubatus")**: This is the nominate subspecies native to Southern Africa and is the most abundant, with an estimated population of more than 6,000 individuals in the wild.

2. **Asiatic Cheetah ("Acinonyx jubatus venaticus")**: This subspecies is critically endangered and is native to Asia.

These subspecies highlight the diversity within the cheetah population, with the South African Cheetah being the most numerous and the Asiatic Cheetah facing significant threats to its survival.

Sources:
page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.'

page_content='The South African Cheetah ("Acinonyx jubatus jubatus"), also known as Namibian Cheetah, is the nominate subspecies of cheetah native to Southern Africa. It is the most abundant subspecies estimated at more than 6,000 individuals in the wild. Since 1990 and onwards, the population was estimated at approximately 2,500 individuals in Namibia, until 2015, the cheetah population has been increased to more than 3,500 in the country.'

page_content='The Asiatic cheetah ("Acinonyx jubatus venaticus") is a critically endangered subspecies of the cheetah native to Asia.'

page_content='The Felinae is a subfamily of the Felidae. It includes sma